In [1]:
from importlib import reload
import sys
from imp import reload
import warnings
import pandas as pd

In [2]:
df = pd.read_csv('../input/imdb_master.csv',encoding="latin-1")
df = df.drop(['Unnamed: 0','type','file'],axis=1)
df.columns = ["review","sentiment"]
df.loc[df['sentiment'] == 'neg', 'sentiment'] = 0
df.loc[df['sentiment'] == 'pos', 'sentiment'] = 1
df = df[df.sentiment != 'unsup']
df

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0
5,"A funny thing happened to me while watching ""M...",0
6,This German horror film has to be one of the w...,0
7,"Being a long-time fan of Japanese film, I expe...",0
8,"""Tokyo Eyes"" tells of a 17 year old Japanese g...",0
9,Wealthy horse ranchers in Buenos Aires have a ...,0


In [3]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(' ')]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text
#for text in df.review:
#    print(clean_text(text))
df['update_review'] = df.review.apply(lambda x: clean_text(x))
def words(df):
    words = set()
    for text in df.update_review:
        words.update(set(text.split(' ')))
    words = dict.fromkeys(words)
    words.pop('')
    return words
W = words(df)
print("pos/neg" in W)

False


In [4]:
def counter(df, words):
    for i in words:
        words[i] = {}
    words['pos/neg'] = {}
    for i in range(len(df)):
        data = df.iloc[i]
        words['pos/neg'][i] = data.sentiment
        text = data.update_review.split(' ')
        for j in text:
            if j!='':
                if i not in words[j]:
                    words[j][i]=1
                else:
                    words[j][i]+=1
        #if (i+1)%500 == 0:
        #    print(i+1)
    result = {}
    for i in words:
        if len(words[i])>1:
            result[i] = words[i]
        #break
    print(len(result))
    return result
words = counter(df, W)
#print(len(words))
#words.pop('')

69977


In [5]:
import math
def TDF(words):
    k = 0
    for i in words:
        if i!='pos/neg':
            k+=1
            #print(len(words[i]))
            j = words[i].keys()
            a = [0 for k in j if words['pos/neg'][k]==0]
            b = len(words[i])-len(a)
            a = len(a)
            #print(a,b)
            #break
            for j in words[i]:
                if a==0:
                    words[i][j] = words[i][j]
                elif b==0:
                    words[i][j] = -words[i][j]
                else:
                    words[i][j] = words[i][j]*math.log(b/a)
            #print(words[i])
            #if k%500==0:
            #    print(words[i])
                #print(k)
    t = words.copy()
    for i in words:
        t[i] = 0
        for j in words[i]:
            t[i]+=words[i][j]
    return t
a = TDF(words)


In [6]:
a.pop('pos/neg')
words={}
for i in a:
    if a[i]!=0:
        words[i]=a[i]
del a
words

{'prudish': 4.176352116579155,
 'hardhitting': 28.345197361464304,
 'trivett': -2,
 'stereotypesarchetypes': -2,
 'scrapping': -4,
 'babettes': 66.54212933375474,
 'preposterous': -165.96068335168633,
 'klara': 11.090354888959125,
 'stockport': 3,
 'witherspoon': 94.4708323390883,
 'philharmonic': 15.567281192442506,
 'embezzles': 2,
 'assy': 24.1415686865115,
 'connectedbr': 4.1588830833596715,
 'upending': -2,
 'forte': -2.0006941552647053,
 'objectify': 2,
 'unsullied': 2.0794415416798357,
 'generatingbr': 2,
 'profoundly': 75.98385550586168,
 'bloodstorm': -2,
 'automobile': 24.851594439501877,
 'exterminator': -19.40812105567846,
 'sadder': -4.1933958062609555,
 'miscasted': -4,
 'pursuer': 12.163953243244928,
 'revelry': 4.1588830833596715,
 'realist': 41.65577714773394,
 'taste': -23.62461939983529,
 'ichikawa': 59.12806248452581,
 'moonchild': 4,
 'pretentious': -556.5935090233313,
 'wurman': -2,
 'psychokiller': -6.931471805599453,
 'dialoguethe': 4.394449154672439,
 'dispropo

In [7]:
len(words)

59677

In [8]:
from torch.utils.data import TensorDataset,DataLoader
import torch
import torchvision

In [10]:
a = pd.Series(words)
a

prudish                    4.176352
hardhitting               28.345197
trivett                   -2.000000
stereotypesarchetypes     -2.000000
scrapping                 -4.000000
babettes                  66.542129
preposterous            -165.960683
klara                     11.090355
stockport                  3.000000
witherspoon               94.470832
philharmonic              15.567281
embezzles                  2.000000
assy                      24.141569
connectedbr                4.158883
upending                  -2.000000
forte                     -2.000694
objectify                  2.000000
unsullied                  2.079442
generatingbr               2.000000
profoundly                75.983856
bloodstorm                -2.000000
automobile                24.851594
exterminator             -19.408121
sadder                    -4.193396
miscasted                 -4.000000
pursuer                   12.163953
revelry                    4.158883
realist                   41

In [ ]:
val_dataset = TensorDataset(a)
#val_ds, test_ds = torch.utils.data.random_split(val_dataset, [1500, 1500])
#val_loader = DataLoader(val_ds, batch_size=128)
#test_loader = DataLoader(test_ds, batch_size=128)